# 7.4 범주형 데이터 이진분류 경진대회 성능 개선 I

- [범주형 데이터 이진분류 경진대회 링크](https://www.kaggle.com/c/cat-in-the-dat/)
- [모델링 코드 참고 링크](https://www.kaggle.com/dkomyagin/cat-in-the-dat-0-80285-private-lb-solution)

In [101]:
import pandas as pd
# 데이터 경로
data_path = '/kaggle/input/cat-in-the-dat/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

## 7.4.1 피처 엔지니어링 I : 피처 맞춤 인코딩

### 데이터 합치기

In [102]:
# 훈련 데이터와 테스트 데이터 합치기
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis=1) # 타깃값 제거

### 이진 피처 인코딩

In [103]:
all_data['bin_3'] = all_data['bin_3'].map({'F':0, 'T':1})
all_data['bin_4'] = all_data['bin_4'].map({'N':0, 'Y':1})

### 순서형 피처 인코딩

In [104]:
ord1dict = {'Novice':0, 'Contributor':1, 
            'Expert':2, 'Master':3, 'Grandmaster':4}
ord2dict = {'Freezing':0, 'Cold':1, 'Warm':2, 
            'Hot':3, 'Boiling Hot':4, 'Lava Hot':5}

all_data['ord_1'] = all_data['ord_1'].map(ord1dict)
all_data['ord_2'] = all_data['ord_2'].map(ord2dict)

In [105]:
from sklearn.preprocessing import OrdinalEncoder

ord_345 = ['ord_3', 'ord_4', 'ord_5']

ord_encoder = OrdinalEncoder() # OrdinalEncoder 객체 생성
# ordinal 인코딩 적용
all_data[ord_345] = ord_encoder.fit_transform(all_data[ord_345])

# 피처별 인코딩 순서 출력
for feature, categories in zip(ord_345, ord_encoder.categories_):
    print(feature)
    print(categories)

ord_3
['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o']
ord_4
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
ord_5
['AP' 'Ai' 'Aj' 'BA' 'BE' 'Bb' 'Bd' 'Bn' 'CL' 'CM' 'CU' 'CZ' 'Cl' 'DH'
 'DN' 'Dc' 'Dx' 'Ed' 'Eg' 'Er' 'FI' 'Fd' 'Fo' 'GD' 'GJ' 'Gb' 'Gx' 'Hj'
 'IK' 'Id' 'JX' 'Jc' 'Jf' 'Jt' 'KR' 'KZ' 'Kf' 'Kq' 'LE' 'MC' 'MO' 'MV'
 'Mf' 'Ml' 'Mx' 'NV' 'Nf' 'Nk' 'OR' 'Ob' 'Os' 'PA' 'PQ' 'PZ' 'Ps' 'QM'
 'Qb' 'Qh' 'Qo' 'RG' 'RL' 'RP' 'Rm' 'Ry' 'SB' 'Sc' 'TR' 'TZ' 'To' 'UO'
 'Uk' 'Uu' 'Vf' 'Vx' 'WE' 'Wc' 'Wv' 'XI' 'Xh' 'Xi' 'YC' 'Yb' 'Ye' 'ZR'
 'ZS' 'Zc' 'Zq' 'aF' 'aM' 'aO' 'aP' 'ac' 'av' 'bF' 'bJ' 'be' 'cA' 'cG'
 'cW' 'ck' 'cp' 'dB' 'dE' 'dN' 'dO' 'dP' 'dQ' 'dZ' 'dh' 'eG' 'eQ' 'eb'
 'eg' 'ek' 'ex' 'fO' 'fh' 'gJ' 'gM' 'hL' 'hT' 'hh' 'hp' 'iT' 'ih' 'jS'
 'jV' 'je' 'jp' 'kC' 'kE' 'kK' 'kL' 'kU' 'kW' 'ke' 'kr' 'kw' 'lF' 'lL'
 'll' 'lx' 'mb' 'mc' 'mm' 'nX' 'nh' 'oC' 'oG' 'oH' 'oK' 'od' 'on' 'pa'
 'ps' 'qA' 'qJ' 'qK' 'qP' 'qX' '

### 명목형 피처 인코딩

In [106]:
nom_features = ['nom_' + str(i) for i in range(10)] # 명목형 피처

In [107]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder() # OneHotEncoder 객체 생성
# 원-핫 인코딩 적용
encoded_nom_matrix = onehot_encoder.fit_transform(all_data[nom_features])

encoded_nom_matrix

<500000x16276 sparse matrix of type '<class 'numpy.float64'>'
	with 5000000 stored elements in Compressed Sparse Row format>

In [108]:
all_data = all_data.drop(nom_features, axis=1) # 기존 명목형 피처 삭제

### 날짜 피처 인코딩

In [109]:
date_features  = ['day', 'month'] # 날짜 피처
all_data_onehot = all_data.copy()
all_data_sprs = all_data.copy()

In [110]:
# # 원-핫 인코딩 적용
# encoded_date_matrix = onehot_encoder.fit_transform(all_data[date_features])
# all_data_onehot = all_data.drop(date_features, axis=1) # 기존 날짜 피처 삭제

# encoded_date_matrix

In [111]:
# 순서 인코딩 적용
from sklearn.preprocessing import OrdinalEncoder

date_features  = ['day', 'month'] # 날짜 피처
ordinal_encoder = OrdinalEncoder() # OrdinalEncoder 객체 생성
encoded_date_matrix = ordinal_encoder.fit_transform(all_data[date_features])
all_data_ord = all_data.drop(date_features, axis=1) # 기존 날짜 피처 삭제

encoded_date_matrix

array([[1., 1.],
       [6., 7.],
       [6., 1.],
       ...,
       [2., 0.],
       [1., 0.],
       [1., 1.]])

## 7.4.2 피처 엔지니어링 II : 피처 스케일링

### 순서형 피처 스케일링

In [112]:
from sklearn.preprocessing import MinMaxScaler

ord_features = ['ord_' + str(i) for i in range(6)] # 순서형 피처
# min-max 정규화
all_data[ord_features] = MinMaxScaler().fit_transform(all_data[ord_features])

### 인코딩 및 스케일링된 피처 합치기

In [113]:
from scipy import sparse

# 인코딩 및 스케일링된 피처 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data),
                               encoded_nom_matrix,
                               encoded_date_matrix],
                              format='csr')

In [114]:
all_data_sprs

<500000x16291 sparse matrix of type '<class 'numpy.float64'>'
	with 9983932 stored elements in Compressed Sparse Row format>

### 데이터 나누기

In [115]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data_sprs[:num_train] # 0 ~ num_train - 1행
X_test = all_data_sprs[num_train:] # num_train ~ 마지막 행

y = train['target']

In [116]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y,
                                                      test_size=0.1,
                                                      stratify=y,
                                                      random_state=10)

## 7.4.3 하이퍼 파라미터 최적화

In [117]:
# %%time

# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LogisticRegression

# # 로지스틱 회귀 모델 생성
# logistic_model = LogisticRegression()

# # 하이퍼파라미터 값 목록
# lr_params = {'C':[0.1, 0.125, 0.2], 'max_iter':[800, 900, 1000], 
#              'solver':['liblinear'], 'random_state':[42]}

# # 그리드서치 객체 생성
# gridsearch_logistic_model = GridSearchCV(estimator=logistic_model,
#                                          param_grid=lr_params,
#                                          scoring='roc_auc', # 평가지표
#                                          cv=5)
# # 그리드서치 수행
# gridsearch_logistic_model.fit(X_train, y_train)

# print('최적 하이퍼파라미터:', gridsearch_logistic_model.best_params_)

num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data_onehot[:num_train] # 0 ~ num_train - 1행
X_test = all_data_onehot[num_train:] # num_train ~ 마지막 행

y = train['target']

from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y,
                                                      test_size=0.1,
                                                      stratify=y,
                                                      random_state=10)

from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 생성
logistic_model = LogisticRegression(
    C=0.1,               # 직접 하이퍼파라미터 지정
    max_iter=1000, 
    solver='liblinear', 
    random_state=42
)

# 모델 학습
logistic_model.fit(X_train, y_train)

LogisticRegression(C=0.1, max_iter=1000, random_state=42, solver='liblinear')

## 7.4.4 모델 성능 검증

In [118]:
# 검증 데이터 예측
y_valid_preds = logistic_model.predict_proba(X_valid)[:, 1]

In [119]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 검증 데이터 ROC AUC
roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

검증 데이터 ROC AUC : 0.7403


## 7.4.5 예측 및 결과 제출

In [121]:
# 타깃값 1일 확률 예측
y_preds = logistic_model.predict_proba(X_test)[:,1]

# 제출 파일 생성
submission['target'] = y_preds
submission.to_csv('ord_submission.csv')

날짜 피처의 성격상 순서가 있기 때문에 순서형 인코딩의 성능이 더 좋을 것 같다. 그러나 원-핫 인코딩의 결과가 더 좋게 나왔다.

![file:///var/folders/fs/ldgbkzq52zng_83v61d49v5c0000gn/T/TemporaryItems/NSIRD_screencaptureui_uCv31p/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202025-09-19%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%209.48.42.png](file:///var/folders/fs/ldgbkzq52zng_83v61d49v5c0000gn/T/TemporaryItems/NSIRD_screencaptureui_uCv31p/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202025-09-19%20%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE%209.48.42.png)